In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import onnxruntime as ort
import numpy as np
import traceback
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader



# Je crée une classe qui représente mon modèle
# Elle hérite de nn.Module qui sert a pytorch pour fonctionné
class CNNModel(nn.Module):
    def __init__(self):
        # J'appelle le constructeur de base
        super(CNNModel, self).__init__()
        #je prépare les images
        # ça va me permettre d'analyser les images d'après ce que j'ai pu comprendre et voir c'est ce que l'on appel une "vision"
        self.conv_layers = nn.Sequential(
            # l'image qui va être passée sera lu par un filtre qui sert à regarder les formes et les motifs
            # le 1 l'image en noir et blanc, 32 le nombre de filtres, kernel_size=3 la taille du filtre, padding=1 pour garder la taille de l'image
            # 28x28 -> 28x28
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  
            # Relu va gardé les valeurs positives et mettre à zéro les négatives
            nn.ReLU(),
            # permet de réduire la taille de l'image par 2 du coup 14x14 il devrait ne garder que les pixels les plus importants 
            nn.MaxPool2d(2),                             
            # Cette partie va continuer  de filtré cette fois-ci avec 64 filtres
            #32 analyse différentes qui va regarder l'image sous différentes formes
            # pour 64 c'est le même principe que pour 32
            nn.Conv2d(32, 64, kernel_size=3, padding=1), 
            # va de nouveau garder les valeurs positives
            nn.ReLU(),
            # Dropout spatial pour les convolutions explication de celui-ci : https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html
            # je le trouve intéressent car il permet de supprimer des canaux entiers temporairement  et ça permet au réseau de neurones de ne pas s'appuyer sur 
            #un seul filtre. du coup certains de 32 canaux seront mis à zéro aléatoirement je pense que ce n'est pas pertinent pour l'excercice mais c'est intéressant
            nn.Dropout2d(0.25),  
            # va de nouveau réduire la taille de l'image par 2 pour ne garder que l essentiel
            nn.MaxPool2d(2)                             
        )

        # la deuxième partie et celle qui va transformer les données filtrées en une forme que le modèle peut utiliser pour faire des prédictions
        self.fc_layers = nn.Sequential(
            # va permettre de mettre les données en une seule ligne pour pouvoir les calculer
            nn.Flatten(),
            # va faire un calcul pour réduire les données à 128 valeurs
            nn.Linear(64 * 7 * 7, 128),
            # va de nouveau garder les valeurs positives
            nn.ReLU(),
            # va aider à éviter le sur-apprentissage en mettant à zéro 50% des valeurs aléatoirement
            nn.Dropout(0.5), 
            # va faire un autre calcul pour réduire les données à 10 valeurs (une pour chaque chiffre de 0 à 9)
            nn.Linear(128, 10)
        )
       # Cette fonction dit comment les données passent à travers le modèle
    def forward(self, x):
        # va pemettre de passer l'image et de lire le contenu
        x = self.conv_layers(x)
        # c'est la partie reçoit les données filtrées et les transforme en une forme que le modèle peut utiliser pour faire des prédictions
        x = self.fc_layers(x)
        # retourne les probabilités  
        return x

# je prépare les images en les transformant en nombres 
transform = transforms.Compose([
    #transforme l'image
    transforms.ToTensor(),
    # ajuste des images pour les rendre plus faciles à traiter
    transforms.Normalize((0.1307,), (0.3081,))  
])

# Je charge les données MNIST
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    #peermet de regarder 64 images à la fois et shuffle=True pour mélanger les images 
    batch_size=64, shuffle=True
)

# Je charge les données de test MNIST
# batch_size=256 pour regarder 256 images à la fois et shuffle=False pour ne pas mélanger les images
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, download=True, transform=transform),
    batch_size=256, shuffle=False
)

# je crée le modèle
model = CNNModel()
#permet de calculer les erreurs
criterion = nn.CrossEntropyLoss()
# permet de mettre à jour les paramètres du modèle pour améliorer les prédictions
# optim.Adam est un algorithme d'optimisation qui ajuste les paramètres du modèle pour améliorer les prédictions
# lr=0.001 est le taux d'apprentissage, qui détermine à quelle vitesse le modèle apprend
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement
# Je vais entraîner le modèle pendant 5 fois
for epoch in range(5):
    # Je mets le modèle en mode entraînement
    # ça permet de dire au modèle qu'il va apprendre et qu'il doit mettre à jour
    model.train()
    ## ça garde une trace des erreurs pendant l'entraînement
    running_loss = 0
    #récupère les images et les valeurs
    for images, labels in train_loader:
        # remet les gradients à zéro pour éviter de les additionner
        optimizer.zero_grad()
        # passe les images dans le modèle pour obtenir les prédictions
        outputs = model(images)
        #comparaison des valeurs
        loss = criterion(outputs, labels)
        # permet de dire les erreurs au modèle pour qu'il puisse apprendre
        loss.backward()
        # ajuste les paramétres du modèle en fonction des erreurs
        # ça permet de mettre à jour les paramètres du modèle pour améliorer les prédictions
        optimizer.step()
        #ajoute l'erreur au total
        running_loss += loss.item()
        # sa affiche l erreur moyenne  plsu elle diminue plus le modéle apprend
    print(f"Époque {epoch+1} terminée. Perte moyenne = {running_loss/len(train_loader):.4f}")

# permet de passer a la prédiction donc ce n'est plus de l'entrainement je vais pouvoir me rendre si l'entrainement a été efficace
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #fait des prédictions sur les images 
        outputs = model(images)
        # ne garde que la valeur la plus proche de la prédiction
        predicted = torch.argmax(outputs, 1)
        # c'est le nombre total d'images
        total += labels.size(0)
        # le nombre de prédictions correctes
        correct += (predicted == labels).sum().item()
        #affiche le taux de réussite
print(f"Précision sur les données de test : {100 * correct / total:.2f}%")

# exportation vers ONNX
# je crée une entrée factice pour le modèle
# ça permet de simuler une image pour l'exportation
dummy_input = torch.randn(1, 1, 28, 28)
# je spécifie le nom du fichier ONNX
onnx_file = "web/mnist_model_cnn.onnx"
# J'exporte le modèle vers le format ONNX
# ça permet de sauvegarder le modèle pour l'utiliser dans d'autres applications
torch.onnx.export(model, dummy_input, onnx_file,
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}})
print(f"Modèle CNN exporté en ONNX dans '{onnx_file}'")

# Test ONNX
# Je charge le modèle ONNX
# ça permet de charger le modèle ONNX pour faire des prédictions
ort_session = ort.InferenceSession(onnx_file)
# Je prends une image de test pour faire une prédiction
test_image = next(iter(test_loader))[0][0].unsqueeze(0).numpy()
# ça permet de préparer l'image pour la prédiction , float32 pour que les valeurs soient en virgule flottante
inputs = {"input": test_image.astype(np.float32)}
# Je fais une prédiction avec le modèle ONNX
outputs = ort_session.run(None, inputs)
# Je prends la valeur la plus proche de la prédiction
# ça permet de savoir quel chiffre le modèle pense que c'est
prediction = np.argmax(outputs[0])
# Affichage de la prédiction
print("Test ONNX : prédiction =", prediction)


# PROJET 2 : RNN NLP            

#J'ai ajouté cette classe pour stopper l'entrainement si le modèle ne s'améliore pas
# car je me suis rendu compte pendant les tests qu'en le laissant continuer a force ça ne ressemble plus a rien
class EarlyStopping:
    # patience est le nombre d'époques sans amélioration avant d'arrêter
    # min_delta est la différence minimale entre la meilleure perte et la perte actuelle pour considérer qu'il y a une amélioration
    def __init__(self, patience=5, min_delta=0):
        # Initialisation des paramètres
        self.patience = patience
        self.min_delta = min_delta
        # Compteur compte combien de fois la performance ne s’est pas améliorée.
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    # Cette méthode est appelée à chaque époque pour vérifier si l'entraînement doit s'arrêter
    def __call__(self, val_loss):
        # Si c'est la première époque, on initialise la meilleure perte
        if self.best_loss is None:
            self.best_loss = val_loss
        # Si la perte ne s’améliore pas suffisamment, on augmente le compteur.
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
           # Si le compteur dépasse la patience, on active l'arrêt anticipé.
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            # Si la perte s’améliore, on la met à jour et on réinitialise le compteur.
            self.best_loss = val_loss
            self.counter = 0

# --- Prétraitement corpus le fichier qui contient une quantité de phrase pour entrainé le modèle---
#va ouvrir le fichier texte, lire tout le contenu, et le mettre en minuscules.
with open("web/corpus.txt", "r", encoding="utf-8") as f:
    raw_text = f.read().lower()

#Supprime tous les caractères non utiles (comme les chiffres ou symboles bizarres).
corpus = re.sub(r"[^a-zàâçéèêëîïôûùüÿ.,;!?'\-\n\r ]", "", raw_text)
#Liste les caractères uniques et crée 2 dictionnaires pour convertir caractères et indices.
chars = sorted(list(set(corpus)))
# permet de convertir les caractères en indices et vice versa
char2idx = {ch: idx for idx, ch in enumerate(chars)}
# idx2char permet de convertir les indices en caractères
idx2char = {idx: ch for ch, idx in char2idx.items()}
# la taille du vocabulaire
vocab_size = len(chars)

print(f"Corpus nettoyé : {len(corpus)} caractères")
print(f"Taille du vocabulaire : {vocab_size} caractères")

# --- Séquences ---
# permet de crée des séquences d'entrée et de sortie pour l'entraînement
# la longueur d’une séquence d’entrée = 10. 
seq_length = 10
# je crée des listes pour stocker les séquences d'entrée et de sortie
input_seqs, target_seqs = [], []

# je parcours le corpus pour créer les séquences d'entrée et de sortie
for i in range(len(corpus) - seq_length):
    # prend une séquence de 10 caractères comme entrée et le caractère suivant comme cible
    input_seq = corpus[i:i + seq_length]
    target_char = corpus[i + seq_length]
    # convertit les caractères en nombre
    # pour chaque caractère de la séquence d'entrée, je le convertis en nombre
    input_seqs.append([char2idx[ch] for ch in input_seq])
    target_seqs.append(char2idx[target_char])

# je convertis les séquences en tenseurs py torch
X = torch.tensor(input_seqs)
y = torch.tensor(target_seqs)

# Split train/val
# je divise les données en ensembles d'entraînement et de validation
# pour que le modèle puisse apprendre sur une partie des données et valider sur une autre
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# --- DataLoaders ---
# le batch_size est le nombre d'exemples traités en même temps
batch_size = 64
# TensorDataset permet de combiner les entrées et les cibles en un seul ensemble de données
train_dataset = TensorDataset(X_train, y_train)
# DataLoader permet de charger les données par lots
# shuffle=True permet de mélanger les données à chaque époque pour éviter l'overfitting
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

#Modèle RNN 
# Je crée une classe pour le modèle RNN
# qui hérite de nn.Module, la classe de base pour tous les modèles pyTorch
class CharRNN(nn.Module):
    # Le constructeur initialise les couches du modèle
    # vocab_size est la taille du vocabulaire, embedding_dim est la dimension des embeddings,
    # hidden_dim est la dimension des états cachés, dropout est le taux de dropout
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout=0.3):
        super(CharRNN, self).__init__()
        # Initialisation des paramètres du modèle
        self.hidden_dim = hidden_dim
        # nn.Embedding permet de transformer les indices des caractères en vecteurs numériques
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # nn.Dropout permet de réduire le sur-apprentissage en mettant à zéro aléatoirement certaines valeurs
        self.dropout = nn.Dropout(dropout)
        # nn.LSTM est une couche LSTM qui permet de traiter les séquences
        # embedding_dim est la dimension des embeddings, hidden_dim est la dimension des états cachés batch_first=True permet de traiter les séquences par lot
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # nn.Linear est une couche linéaire qui permet de transformer les états cachés en prédictions
        self.fc = nn.Linear(hidden_dim, vocab_size)

    # La méthode forward définit comment les données passent à travers le modèle
    # x est la séquence d'entrée, h est l'état caché (optionnel)
    # si h est None, j' initialise l'état caché à zéro
    def forward(self, x, h=None):
        x = self.embedding(x)
        x = self.dropout(x)
        if h is None:
            out, h = self.rnn(x)
        else:
            out, h = self.rnn(x, h)
        # out est la sortie de la couche RNN, h est l'état caché final
        # j applique le dropout pour réduire le sur-apprentissage
        out = self.dropout(out)
        # je prends la dernière sortie de la séquence pour la prédiction
        # out[:, -1, :] prend la dernière sortie de chaque séquence dans le lot
        out_fc = self.fc(out[:, -1, :])
        # retourne la sortie finale et l'état caché
        return out_fc, h  

#Hyperparams
# Je définis les hyperparamètres du modèle
#embedding_dim est la dimension des embeddings, hidden_dim est la dimension des états cachés,
# num_epochs est le nombre d'époques pour l'entraînement
embedding_dim = 64
hidden_dim = 512
num_epochs = 60

# Je crée une instance du modèle RNN
model = CharRNN(vocab_size, embedding_dim, hidden_dim)
#la fonction de perte pour compare la prédiction au bon caractère.
criterion = nn.CrossEntropyLoss()
# optim.Adam est un algorithme d'optimisation qui ajuste les paramètres du modèle pour améliorer les prédictions
# lr=0.001 est le taux d'apprentissage ici j 'ai essayé avec différentes valeurs, qui détermine à quelle vitesse le modèle apprend
# weight_decay=1e-4 est une pénalité pour éviter le sur-apprentissage
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
# je crée un planificateur de taux d'apprentissage pour réduire le taux d'apprentissage si la perte de validation ne s'améliore pas
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
# J'instancie la classe EarlyStopping pour arrêter l'entraînement si la perte de validation ne s'améliore pas
# patience=5 signifie que l'entraînement s'arrêtera si la perte de validation n'a pas d amélioration pendant 5 époques
early_stopping = EarlyStopping(patience=5)


#Entraînement
# je crée une boucle d'entraînement pour entraîner le modèle
# pour chaque époque, je mets le modèle en mode entraînement, je calcule la perte
for epoch in range(num_epochs):
    # Je mets le modèle en mode entraînement
    model.train()
    train_loss_total = 0

    # Je parcours le DataLoader d'entraînement pour obtenir les lots de données
    # xb est le lot d'entrées, yb est le lot de cibles
    for xb, yb in train_loader:
        output, _ = model(xb)
        loss = criterion(output, yb)

        # Je remets les gradients à zéro, calcule la perte, et mets à jour les paramètres du modèle
        # ça permet de ne pas additionner les gradients des lots précédents
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # J'additionne la perte totale pour l'entraînement
        # loss.item() retourne la valeur de la perte pour le lot actuel
        train_loss_total += loss.item() * xb.size(0)
    train_loss = train_loss_total / len(train_dataset)

    # je mets le mode évaluation.
    model.eval()
    val_loss_total = 0
    # Je parcours le DataLoader de validation pour obtenir les lots de données
    # avec torch.no_grad() pour ne pas calculer les gradients pendant la validation
    with torch.no_grad():
        for xb, yb in val_loader:
            val_output, _ = model(xb)
            val_loss_total += criterion(val_output, yb).item() * xb.size(0)
    val_loss = val_loss_total / len(val_dataset)

    print(f"[RNN] Époque {epoch+1}/{num_epochs} - Perte train : {train_loss:.4f} - Val : {val_loss:.4f}")

    # j'ajuste le learning rate et on vérifie si on doit arrêter.
    scheduler.step(val_loss)
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping déclenché.")
        break

#Export ONNX
onnx_file = "web/rnn_text_gen.onnx"
os.makedirs(os.path.dirname(onnx_file), exist_ok=True)

model.eval()
model.cpu()

# Je crée une entrée factice pour l'exportation
# ça permet de simuler une séquence d'entrée pour l'exportation
dummy_input = torch.randint(0, vocab_size, (1, seq_length)).long()
# Je crée des états cachés initiaux pour l'exportation
# h0 et c0 sont les états cachés initiaux pour la couche LSTM
h0 = torch.zeros(1, 1, hidden_dim)
c0 = torch.zeros(1, 1, hidden_dim)

try:
    #convertit le modèle en format ONNX pour être utilisé ailleurs comme un navigateur.
    torch.onnx.export(
        model,
        (dummy_input, (h0, c0)),
        onnx_file,
        input_names=["input", "h0", "c0"],
        output_names=["output", "hn", "cn"],
        dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
        opset_version=11
    )
    print(f"Modèle RNN exporté en ONNX vers '{onnx_file}'")
except Exception as e:
    print("Échec de l’export ONNX :", e)

print("📁 Fichier ONNX généré ?", os.path.exists(onnx_file))

# Test rapide ONNX
#Fait 5 prédictions aléatoires avec le modèle ONNX, et j'affiche le caractère prédit.
try:
    ort_session = ort.InferenceSession(onnx_file)
    print("Test de 5 prédictions ONNX aléatoires :")
    for _ in range(5):
        test_input = torch.randint(0, vocab_size, (1, seq_length)).long().numpy()
        ort_inputs = {"input": test_input}
        ort_output = ort_session.run(None, ort_inputs)
        predicted_idx = np.argmax(ort_output[0])
        predicted_char = idx2char.get(int(predicted_idx), "?")
        input_str = ''.join([idx2char[i] for i in test_input[0]])
        print(f"Input: '{input_str}' ➜ Pred: '{predicted_char}'")
except Exception as e:
    print("Erreur ONNX :", e)

# Sauvegarde vocab
#enregistre les dictionnaires char2idx et idx2char dans un fichier JSON.
vocab_path = "web/vocab.json"
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump({
        "char2idx": char2idx,
        "idx2char": {str(k): v for k, v in idx2char.items()}
    }, f, ensure_ascii=False)
print(f"Vocabulaire sauvegardé dans {vocab_path}")

# Génération de texte
# Prend une probabilité et choisit un caractère au hasard, influencé par la température.
def sample_from_probs(probs, temperature=1.0, top_k=None):
    probs = probs / temperature
    # Exponentiation pour obtenir des probabilités
    probs = torch.exp(probs)
    # Si top_k est spécifié, on ne garde que les k meilleures probabilités
    if top_k is not None:
        top_k = min(top_k, probs.size(0))  
        top_k_probs, top_k_indices = torch.topk(probs, top_k)
        top_k_probs = top_k_probs / torch.sum(top_k_probs)
        next_idx = torch.multinomial(top_k_probs, 1).item()
        return top_k_indices[next_idx].item()
    else:
        probs = probs / torch.sum(probs)
        next_idx = torch.multinomial(probs, 1).item()
        return next_idx

#Utilise le modèle PyTorch pour générer du texte à partir d’un début.
#Cette fonction prend un texte de départ, le convertit en indices, et génère du texte caractère par caractère.
def generate_text(model, start_text, char2idx, idx2char, seq_length=10, max_length=100, temperature=0.7, top_k=10):
    model.eval()
    # Commence par le texte de départ
    generated = start_text
    # Convertit le texte de départ en indices, en s'assurant qu'il a la bonne longueur
    # rjust permet de compléter le texte de départ avec des espaces pour atteindre la longueur requise
    input_seq = [char2idx.get(ch, 0) for ch in start_text[-seq_length:].rjust(seq_length)]
    # Convertit la séquence d'entrée en tenseur PyTorch
    # unsqueeze(0) ajoute une dimension pour le batch size
    input_seq = torch.tensor(input_seq).unsqueeze(0)  

    # Initialise l'état caché à None pour la première génération
    # h est l'état caché de la couche LSTM, il sera mis à jour
    h = None  

    # Boucle pour générer du texte caractère par caractère
    with torch.no_grad():
        for _ in range(max_length):
            output, h = model(input_seq, h)  
            probs = F.log_softmax(output, dim=1).squeeze() 
            next_idx = sample_from_probs(probs, temperature, top_k=top_k)
            next_char = idx2char.get(next_idx, '?')
            generated += next_char

            input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[next_idx]])], dim=1)

    return generated


# Génération de texte avec ONNX
# Cette fonction utilise le modèle ONNX pour générer du texte à partir d’un début.
## Elle prend un texte de départ, le convertit en indices, et génère du texte caractère par caractère.
# Elle utilise onnxruntime pour exécuter le modèle ONNX.
def generate_text_onnx(ort_session, start_text, char2idx, idx2char, seq_length=10, max_length=100, temperature=1.0):
    #je commence le texte généré avec le texte de départ start_text. C’est la base sur laquelle le modèle va continuer à écrire.
    generated = start_text
    # je convertis les derniers caractères de start_text juste la fin, selon seq_length en nombres à l’aide du dictionnaire char2idx.
    input_seq = [char2idx.get(ch, 0) for ch in start_text[-seq_length:].rjust(seq_length)]
    # Je transforme cette séquence d'indices en un tableau NumPy pour l'entrée du modèle ONNX.
    # reshape(1, -1) permet de s'assurer que la séquence a la bonne forme pour le modèle.
    # astype(np.int64) convertit les indices en entiers 64 bits,
    input_seq = np.array(input_seq).reshape(1, -1).astype(np.int64)

    # Initialise les états cachés à zéro pour la première génération
    # h0 et c0 sont les états cachés initiaux pour la couche LSTM
    h0 = np.zeros((1, 1, hidden_dim), dtype=np.float32)
    c0 = np.zeros((1, 1, hidden_dim), dtype=np.float32)

    # Boucle pour générer du texte caractère par caractère
    for _ in range(max_length):
        #la séquence actuelle est passée au modèle ONNX pour obtenir les probabilités de sortie.
        # ort_inputs est un dictionnaire qui contient les entrées du modèle ONNX.
        ort_inputs = {"input": input_seq, "h0": h0, "c0": c0}
        # Exécute le modèle ONNX pour obtenir les sorties
        # ort_output contient les sorties du modèle ONNX, qui sont les probabilités de chaque caractère.
        # Il contient aussi les nouveaux états cachés h0 et c0 pour la prochaine itération.
        # ort_session.run None, ort_inputs exécute le modèle ONNX avec les entrées fournies.
        # None signifie que je veux toutes les sorties du modèle.
        ort_output = ort_session.run(None, ort_inputs)
        #je récupère la probabilité prédite pour le prochain caractère.
        output_probs = ort_output[0][0]
        h0, c0 = ort_output[1], ort_output[2] 

        # Je convertis les probabilités de sortie en un tenseur PyTorch pour appliquer softmax.
        # torch.tensor transforme la liste de probabilités en un tenseur PyTorch.
        probs = torch.tensor(output_probs)
        probs = F.softmax(probs, dim=0)
        # Je sample un indice de caractère à partir des probabilités, influencé par la température.
        # sample_from_probs est une fonction qui prend les probabilités et la température pour choisir un caractère.
        next_idx = sample_from_probs(probs, temperature)
        #permet de coinvertir l'index du caractère choisi next_idx en lettre avec le dictionnaire idx2char.
        next_char = idx2char.get(next_idx, '?')
        generated += next_char
        input_seq = np.concatenate([input_seq[:, 1:], np.array([[next_idx]])], axis=1)

    return generated


# Exemples de génération
if __name__ == "__main__":
    # C’est le début du texte à partir duquel on va générer la suite.
    start_text = "bonjour je"
    # je vais tester plusieurs températures pour la génération Température basse ex : 0.5 rend le modèle plus prévisible, moins créatif.
    #température haute ex : 1.2 rend le modèle plus aléatoire, parfois plus original mais aussi plus instable.
    temperatures = [0.5, 0.8, 1.2]

    # J'affiche les résultats de la génération de texte pour chaque température
    for temp in temperatures:
        print(f"\n--- 🔮 Température : {temp} ---")
        print("PyTorch :", generate_text(model, start_text, char2idx, idx2char, seq_length, max_length=100, temperature=temp))
        try:
            print("ONNX :", generate_text_onnx(ort_session, start_text, char2idx, idx2char, seq_length, max_length=100, temperature=temp))
        except Exception as e:
            print("Erreur génération ONNX :", e)

Époque 1 terminée. Perte moyenne = 0.2358
Époque 2 terminée. Perte moyenne = 0.0926
Époque 3 terminée. Perte moyenne = 0.0739
Époque 4 terminée. Perte moyenne = 0.0628
Époque 5 terminée. Perte moyenne = 0.0530
Précision sur les données de test : 99.21%
Modèle CNN exporté en ONNX dans 'web/mnist_model_cnn.onnx'
Test ONNX : prédiction = 7
🧹 Corpus nettoyé : 2953 caractères
🔠 Taille du vocabulaire : 38 caractères
[RNN] Époque 1/60 - Perte train : 2.9408 - Val : 2.5990
[RNN] Époque 2/60 - Perte train : 2.4612 - Val : 2.3831
[RNN] Époque 3/60 - Perte train : 2.2691 - Val : 2.3313
[RNN] Époque 4/60 - Perte train : 2.1486 - Val : 2.2221
[RNN] Époque 5/60 - Perte train : 2.0316 - Val : 2.1899
[RNN] Époque 6/60 - Perte train : 1.9317 - Val : 2.1808
[RNN] Époque 7/60 - Perte train : 1.8519 - Val : 2.1307
[RNN] Époque 8/60 - Perte train : 1.7485 - Val : 2.1019
[RNN] Époque 9/60 - Perte train : 1.6602 - Val : 2.1071
[RNN] Époque 10/60 - Perte train : 1.5259 - Val : 2.1651
[RNN] Époque 11/60 - Pert